## Saturday, November 11, 2023

https://blog.futuresmart.ai/integrating-llama-2-with-hugging-face-and-langchain

Gonna give this a go ...

This sample page fails right away on the PromptTemplate stuff ... sigh.

In [1]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer=AutoTokenizer.from_pretrained(model)

In [2]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
%%time
sequences = pipeline(
    'Hi! I like cooking. Can you suggest some recipes?\n')
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

    # 2m 5.8s


Result: Hi! I like cooking. Can you suggest some recipes?

Hey! I'm glad you're interested in cooking. Here are some delicious recipes you might enjoy trying:

1. Chicken Parmesan - breaded and fried chicken topped with marinara sauce and melted mozzarella cheese, served with pasta.
2. Beef Stroganoff - sautéed beef strips in a creamy sauce made with sour cream, Dijon mustard, and mushrooms, served over egg noodles.
3. Vegetable stir-fry - a colorful medley of sautéed vegetables (such as bell peppers, carrots, broccoli, and onions) cooked in a wok with a savory sauce.
4. Baked salmon - a healthy and flavorful dish made by seasoning salmon fillets with lemon juice, salt, and pepper, and baking them in the oven until flaky and tender.
5. Quiche Lorraine - a classic French dish made with eggs, cream, and cheese, baked in a pie crust until golden and puffed.

I hope these suggestions inspire you to try some new recipes! Do you have any dietary restrictions or preferences I should take into

Using Langchain

In [4]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')


In [5]:
# model="meta-llama/Llama-2-7b-chat-hf"
# tokenizer=AutoTokenizer.from_pretrained(model)
# pipeline=transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     device_map="auto",
#     max_length=1000,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id
#     )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})


Defining the Prompt

In [7]:
# <s>[INST] <<SYS>>
# {{ system_prompt }}
# <</SYS>>

# {{ user_message }} [/INST]

In [10]:
from langchain.prompts import PromptTemplate


In [11]:
prompt_template = """<s>[INST] <<SYS>>
{{ You are a helpful AI Assistant}}<<SYS>>
###

Previous Conversation:
'''
{history}
'''

{{{input}}}[/INST]

"""
prompt = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])


In [12]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [13]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

In [15]:
from langchain_experimental.llms import ChatLlamaAPI

ModuleNotFoundError: No module named 'langchain_experimental'

In [14]:
chain = ConversationChain(llm=llm, prompt=prompt)


ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects ['product'], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)